In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Other imports for processing data
import string
import numpy as np
import pandas as pd

In [2]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


In [3]:
!python -m wget https://drive.google.com/uc?id=1LiJFZd41ofrWoBtW-pMYsfz1w8Ny0Bj8 -o songdata.csv


Saved under songdata (2).csv


In [4]:
def tokenize_corpus(corpus, num_words=-1):
  # Fit a Tokenizer on the corpus
  if num_words > -1:
    tokenizer = Tokenizer(num_words=num_words)
  else:
    tokenizer = Tokenizer()
  tokenizer.fit_on_texts(corpus)
  return tokenizer

def create_lyrics_corpus(dataset, field):
  # Remove all other punctuation
  dataset[field] = dataset[field].str.replace('[{}]'.format(string.punctuation), '')
  # Make it lowercase
  dataset[field] = dataset[field].str.lower()
  # Make it one long string to split by line
  lyrics = dataset[field].str.cat()
  corpus = lyrics.split('\n')
  # Remove any trailing whitespace
  for l in range(len(corpus)):
    corpus[l] = corpus[l].rstrip()
  # Remove any empty lines
  corpus = [l for l in corpus if l != '']

  return corpus

In [5]:
# Read the dataset from csv - just first 10 songs for now
dataset = pd.read_csv('songdata.csv', dtype=str)[:10]
# Create the corpus using the 'text' column containing lyrics
corpus = create_lyrics_corpus(dataset, 'text')
# Tokenize the corpus
tokenizer = tokenize_corpus(corpus)

total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

{'you': 1, 'i': 2, 'and': 3, 'a': 4, 'me': 5, 'the': 6, 'is': 7, 'my': 8, 'to': 9, 'ma': 10, 'it': 11, 'of': 12, 'im': 13, 'your': 14, 'love': 15, 'so': 16, 'as': 17, 'that': 18, 'in': 19, 'andante': 20, 'boomaboomerang': 21, 'make': 22, 'on': 23, 'oh': 24, 'for': 25, 'but': 26, 'new': 27, 'bang': 28, 'its': 29, 'be': 30, 'like': 31, 'know': 32, 'now': 33, 'how': 34, 'could': 35, 'youre': 36, 'sing': 37, 'never': 38, 'no': 39, 'chiquitita': 40, 'can': 41, 'we': 42, 'song': 43, 'had': 44, 'good': 45, 'youll': 46, 'she': 47, 'just': 48, 'girl': 49, 'again': 50, 'will': 51, 'take': 52, 'please': 53, 'let': 54, 'am': 55, 'eyes': 56, 'was': 57, 'always': 58, 'cassandra': 59, 'blue': 60, 'time': 61, 'dont': 62, 'were': 63, 'return': 64, 'once': 65, 'then': 66, 'sorry': 67, 'cryin': 68, 'over': 69, 'feel': 70, 'ever': 71, 'believe': 72, 'what': 73, 'do': 74, 'go': 75, 'all': 76, 'out': 77, 'think': 78, 'every': 79, 'leave': 80, 'look': 81, 'at': 82, 'way': 83, 'one': 84, 'music': 85, 'down': 

C:\Users\Admin\AppData\Local\Temp/ipykernel_7884/2906562814.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset[field] = dataset[field].str.replace('[{}]'.format(string.punctuation), '')


In [6]:
sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		sequences.append(n_gram_sequence)

# Pad sequences for equal input length 
max_sequence_len = max([len(seq) for seq in sequences])
sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))

# Split sequences between the "input" sequence and "output" predicted word
input_sequences, labels = sequences[:,:-1], sequences[:,-1]
# One-hot encode the labels
one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [7]:
# Check out how some of our data is being stored
# The Tokenizer has just a single index per word
print(tokenizer.word_index['know'])
print(tokenizer.word_index['feeling'])
# Input sequences will have multiple indexes
print(input_sequences[5])
print(input_sequences[6])
# And the one hot labels will be as long as the full spread of tokenized words
print(one_hot_labels[5])
print(one_hot_labels[6])

32
97
[  0   0   0   0   0   0   0   0   0   0   0   0   0  81  82 142 197  29
   4]
[  0   0   0   0   0   0   0   0   0   0   0   0  81  82 142 197  29   4
 287]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(input_sequences, one_hot_labels, epochs=200, verbose=1)

Epoch 1/200
62/62 [==============================] - 3s 9ms/step - loss: 5.9941 - accuracy: 0.0283
Epoch 2/200
62/62 [==============================] - 1s 9ms/step - loss: 5.4366 - accuracy: 0.0399
Epoch 3/200
62/62 [==============================] - 1s 9ms/step - loss: 5.3700 - accuracy: 0.0399
Epoch 4/200
62/62 [==============================] - 1s 9ms/step - loss: 5.3162 - accuracy: 0.0399
Epoch 5/200
62/62 [==============================] - 1s 9ms/step - loss: 5.2489 - accuracy: 0.0363
Epoch 6/200
62/62 [==============================] - 1s 9ms/step - loss: 5.1854 - accuracy: 0.0404
Epoch 7/200
62/62 [==============================] - 1s 9ms/step - loss: 5.1303 - accuracy: 0.0404
Epoch 8/200
62/62 [==============================] - 1s 9ms/step - loss: 5.0735 - accuracy: 0.0399
Epoch 9/200
62/62 [==============================] - 1s 9ms/step - loss: 5.0034 - accuracy: 0.0404
Epoch 10/200
62/62 [==============================] - 1s 9ms/step - loss: 4.9395 - accuracy: 0.0505
Epoch 11/

62/62 [==============================] - 1s 9ms/step - loss: 1.5452 - accuracy: 0.6766
Epoch 84/200
62/62 [==============================] - 1s 9ms/step - loss: 1.5170 - accuracy: 0.6786
Epoch 85/200
62/62 [==============================] - 1s 9ms/step - loss: 1.4997 - accuracy: 0.6806
Epoch 86/200
62/62 [==============================] - 1s 9ms/step - loss: 1.4876 - accuracy: 0.6922
Epoch 87/200
62/62 [==============================] - 1s 9ms/step - loss: 1.4511 - accuracy: 0.6963
Epoch 88/200
62/62 [==============================] - 1s 9ms/step - loss: 1.4328 - accuracy: 0.6973
Epoch 89/200
62/62 [==============================] - 1s 9ms/step - loss: 1.4104 - accuracy: 0.7074
Epoch 90/200
62/62 [==============================] - 1s 9ms/step - loss: 1.3940 - accuracy: 0.7079
Epoch 91/200
62/62 [==============================] - 1s 9ms/step - loss: 1.3918 - accuracy: 0.7099
Epoch 92/200
62/62 [==============================] - 1s 9ms/step - loss: 1.3759 - accuracy: 0.7079
Epoch 93/200


62/62 [==============================] - 1s 9ms/step - loss: 0.6049 - accuracy: 0.8628
Epoch 165/200
62/62 [==============================] - 1s 9ms/step - loss: 0.5962 - accuracy: 0.8628
Epoch 166/200
62/62 [==============================] - 1s 9ms/step - loss: 0.5860 - accuracy: 0.8663
Epoch 167/200
62/62 [==============================] - 1s 9ms/step - loss: 0.5894 - accuracy: 0.8633
Epoch 168/200
62/62 [==============================] - 1s 9ms/step - loss: 0.5839 - accuracy: 0.8648
Epoch 169/200
62/62 [==============================] - 1s 9ms/step - loss: 0.5724 - accuracy: 0.8683
Epoch 170/200
62/62 [==============================] - 1s 9ms/step - loss: 0.5682 - accuracy: 0.8648
Epoch 171/200
62/62 [==============================] - 1s 9ms/step - loss: 0.5589 - accuracy: 0.8724
Epoch 172/200
62/62 [==============================] - 1s 9ms/step - loss: 0.5850 - accuracy: 0.8592
Epoch 173/200
62/62 [==============================] - 1s 9ms/step - loss: 0.5841 - accuracy: 0.8663
Epoc

In [10]:
seed_text = "im feeling chills"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 17ms/step
im feeling chills me down in our they mean so much take on on on is dead gone a so break is mistake our laughter and it slow and make me sound and it so on on park shining more above you me to stay before on together scars been so on on is is is behind my shouting shouting leave me girl me girl scars pain pain ways pain evening evening evening evening evening evening evening evening evening evening sure scars girl feet scars girl had power power power power power power power power power power power power final scars scars too


In [12]:
# Use this process for the full output generation
seed_text = "im feeling chills"
next_words = 100
  
for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted_probs = model.predict(token_list)[0]
  predicted = np.random.choice([x for x in range(len(predicted_probs))],
                               p=predicted_probs)
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 18ms/step
im feeling chills me down alone so cold like i do what could that she could be on in my wait alone cause is is is all my dreams on play wanted other slowly sorrow thank so quiet blue could on shining play good restrained blue kisses on deal beg good darling song there is no shining in shouting special to me down on play a new my feet none we seems ways front love is before good kiss is here will could you together you me girl bone last break on good every break is way once once again and dancing on
